In [10]:
from physkit.elements import ELEMENTS
from physkit.constants import ConstantsSI
from physkit.units import Mass
from physkit.units import Pressure, Temperature
from vapor_pressure import VaporPressureCurve
from vapor_pressure import VaporPressureAntoineCurve
import numpy as np

In [11]:
Al_g_per_mol = ELEMENTS["Al"].mass
Al_kg_per_mol = Mass.convert(
  from_=(Al_g_per_mol, Mass.Units.g),
  to=Mass.Units.kg)
Al_mass_kg = Al_kg_per_mol/ConstantsSI.N_A
Al_vp: VaporPressureCurve = VaporPressureAntoineCurve(
    P_unit=Pressure.Units.Pa,                 # public metadata
    T_unit=Temperature.Units.K,               # public input unit
    A=5.73623,
    B=13204.109,
    C=-24.306,
    valid_range_K=(1557.0, 2329.0),
    source="NIST WebBook; Stull (1947)",
    P_unit_native=Pressure.Units.bar,         # coefficient-table pressure unit
    T_unit_native=Temperature.Units.K,         # coefficient-table temperature unit
)


In [12]:
Cu_g_per_mol = ELEMENTS["Cu"].mass
Cu_kg_per_mol = Mass.convert(
  from_=(Cu_g_per_mol, Mass.Units.g),
  to=Mass.Units.kg)
Cu_mass_kg = Cu_kg_per_mol/ConstantsSI.N_A
# https://www.scribd.com/document/916585744/Antoine-Coefficients-for-Vapor-Pressure-of-the-Elements#:~:text=of%20The%20Elements-,The%20document%20presents%20a%20comprehensive%20tabulation%20of%20vapor%20pressure%20data,CAS%20numbers%2C%20and%20Antoine%20coefficients.

Cu_vp: VaporPressureCurve = VaporPressureAntoineCurve(
    P_unit=Pressure.Units.Pa,                 # public metadata
    T_unit=Temperature.Units.K,               # public input unit
    A=8.310,
    B=1.5007e4,
    C=-202.14,
    valid_range_K=(
      Temperature.convert(
        from_=(1216.0,Temperature.Units.C), 
        to=Temperature.Units.K),
      Temperature.convert(
        from_=(2562.0,Temperature.Units.C), 
        to=Temperature.Units.K)
    ),
    source="NIST WebBook; Stull (1947)",
    P_unit_native=Pressure.Units.mmHg,         # coefficient-table pressure unit
    T_unit_native=Temperature.Units.C,         # coefficient-table temperature unit
)



For a binary $A-B$ binary solution, the molar excess Gibbs free energy is
$$
  G^\text{ex} = \Omega x_\text{A} x_\text{B}
$$
where
- $x_\text{A}$, $x_\text{B}$: mole fractions of $A$ and $B$, respectively
- $x_\text{A} + x_\text{B} = 1$
- $[\Omega]=\mathrm{J/mol}$

#### Connection to bond energy
$$
  \Omega 
  = 
  z 
  N_A 
  \left(
    \varepsilon_\text{AB} 
    - \frac{\varepsilon_\text{AA}+\varepsilon_\text{BB}}{2}
  \right)
$$
where
- $z$: the coordination number,
- $N_A$: Avogadro's number
- $\varepsilon_{ij}$: bond energy between species $i$ and $j$

#### Connection to Enthalpy of Mixing
The enthalpy of mixing per mole is
$$
  \Delta H_\text{mix} = \Omega x_\text{A} x_\text{B}
$$

#### Connection to activity levels
$$\begin{align}
  \ln \gamma_\text{A} 
  &= \frac{1}{R_\text{g}T}
    \left(
      \frac{\partial G^\text{ex}}{\partial n_\text{A}}
    \right)_{T,P,n_B} \\
  &= \frac{\Omega}{R_\text{g} T} x_\text{B}^2
\end{align}$$

Therefore,
$$\begin{gather}
\ln \gamma_\text{A}=\frac{\Omega}{R_\text{g} T} x_\text{B}^2 \\
\ln \gamma_\text{B}=\frac{\Omega}{R_\text{g} T} x_\text{A}^2
\end{gather}$$

and
$$\begin{gather}

\end{gather}$$

In [ ]:
pure_vp = {
  "Al": Al_vp,
  "Cu": Cu_vp
}

def omega_to_gamma(
    *,
    Omega_J_per_mol: float,
    T: float,
    x_A: float,
    R: float = ConstantsSI.R_g
):
    if not (0.0 <= x_A <= 1.0):
        raise ValueError("x must be between 0 and 1")
    if T <= 0:
        raise ValueError("T must be > 0 K")
    
    x_B = 1.0 - x_A

    ln_gamma_A = Omega_J_per_mol / (R * T) * x_B**2
    ln_gamma_B = Omega_J_per_mol / (R * T) * x_A**2

    return np.exp(ln_gamma_A), np.exp(ln_gamma_B)

  

from mixtures import IdealSolution
from mixtures import PartialPressureFromActivity
from mixtures import RegularSolutionBinary
# ideal solution
pp_ideal = PartialPressureFromActivity(
  pure_vp=pure_vp,
  activity=IdealSolution(species=("Al","Cu"))
)

# regular solution
pp_reg = PartialPressureFromActivity(
    pure_vp=pure_vp,
    activity=RegularSolutionBinary(
      species=("Al","Cu"), 
      Omega_J_per_mol=25_000.0)
)


x = {"Al": 0.8, "Cu": 0.2}
T = 1800.0

Pi_ideal = pp_ideal.P_i_Pa(T, x)
Pi_reg   = pp_reg.P_i_Pa(T, x)

print("Ideal:", Pi_ideal, "Ptot", pp_ideal.P_total_Pa(T,x))
print("Reg  :", Pi_reg,   "Ptot", pp_reg.P_total_Pa(T,x))



TypeError: unsupported format string passed to dict.__format__